<div class="markdown-google-sans">
  <h1>Colab へようこそ</h1>
</div>

<div class="markdown-google-sans">
  <h2>（新規）Gemini API をお試しください</h2>
  <ul>
  <li><a href="https://makersuite.google.com/app/apikey">Generate a Gemini API key</a></li>
  <li><a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/Talk_to_Gemini_with_Google%27s_Speech_to_Text_API.ipynb?utm_medium=link&utm_campaign=gemini">Talk to Gemini with the Speech-to-Text API</a></li>
  <li><a href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb?utm_medium=link&utm_campaign=gemini">Gemini API: Quickstart with Python</a></li>
  <li><a href="https://colab.research.google.com/notebooks/snippets/gemini.ipynb?utm_medium=link&utm_campaign=gemini">Gemini API code sample</a></li>
  <li><a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/Learning_with_Gemini_and_ChatGPT.ipynb?utm_medium=link&utm_campaign=gemini">Compare Gemini with ChatGPT</a></li>  
  <li><a href="https://colab.google/notebooks/?utm_medium=link&utm_campaign=gemini">More notebooks</a></li>
  </ul>
</div>


# データ加工してダウンロード

# ファイルを読み込む

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.tsv to train.tsv


In [26]:
import pandas as pd

# Assuming the file name is 'data.tsv'
df = pd.read_csv('train.tsv', sep='\t')

# データの確認

In [27]:
print(df.head())

   id  class buying maint  doors persons lug_boot safety
0   0  unacc    low   med      3       2    small    low
1   3    acc    low  high      3    more    small    med
2   7  unacc  vhigh  high  5more       2    small    med
3  11    acc   high  high      3    more      big    med
4  12  unacc   high  high      3       2      med   high


# 特定のカラムを変換:(5カラム目と6カラム目に含まれる"5more"や"more"を整数の5に変換します。)

In [28]:
# ユニークな値の置換を設定:
mapping = {'5more': 5, 'more': 5, 'med': 3}

# 安全な変換を実行:
df.iloc[:, 4] = df.iloc[:, 4].replace(mapping).astype(int, errors='ignore')
df.iloc[:, 5] = df.iloc[:, 5].replace(mapping).astype(int, errors='ignore')

# 変換されなかった値の処理:
print(df[df.iloc[:, 5].isna()])
print(df[df.iloc[:, 6].isna()])

Empty DataFrame
Columns: [id, class, buying, maint, doors, persons, lug_boot, safety]
Index: []
Empty DataFrame
Columns: [id, class, buying, maint, doors, persons, lug_boot, safety]
Index: []


# 置換データの確認:

In [29]:
print(df.head())

   id  class buying maint doors persons lug_boot safety
0   0  unacc    low   med     3       2    small    low
1   3    acc    low  high     3       5    small    med
2   7  unacc  vhigh  high     5       2    small    med
3  11    acc   high  high     3       5      big    med
4  12  unacc   high  high     3       2      med   high


# データをシャッフルし、分割する:

In [30]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 読み込んだデータを確認する:

In [31]:
print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))

Training set size: 691
Validation set size: 173


# データをTSVファイルとして保存

In [32]:
train_df.to_csv('train_8.tsv', sep='\t', index=False)
val_df.to_csv('train_2.tsv', sep='\t', index=False)

# ローカルにダウンロード

In [34]:
from google.colab import files

# トレーニングデータのダウンロード
files.download('train_8.tsv')

# 検証データのダウンロード
files.download('train_2.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 決定木使用

In [39]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

# データの読み込み
train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1行目が不要なら削除（場合によって削除が必要）
train_data = train_data.drop(index=0).reset_index(drop=True)
val_data = val_data.drop(index=0).reset_index(drop=True)

# 説明変数と目的変数の分割
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 必要に応じて、分類レポートや混同行列の出力も可能
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_val, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, predictions))

Saving train_2.tsv to train_2 (1).tsv
Accuracy: 95.38%

Classification Report:
              precision    recall  f1-score   support

         acc       0.91      0.93      0.92        43
        good       0.75      0.43      0.55         7
       unacc       0.98      0.99      0.99       119
       vgood       0.80      1.00      0.89         4

    accuracy                           0.95       173
   macro avg       0.86      0.84      0.84       173
weighted avg       0.95      0.95      0.95       173


Confusion Matrix:
[[ 40   1   2   0]
 [  3   3   0   1]
 [  1   0 118   0]
 [  0   0   0   4]]


# r3_特徴量の型が混在しているので一貫化　結果はおかしくなった

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1カラム目を削除（不要な情報が含まれている場合）
train_data = train_data.drop(columns=0)
val_data = val_data.drop(columns=0)

# 説明変数と目的変数の分割（目的変数が新たに2カラム目となるので、列インデックス0）
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# カラム名をすべて文字列に変換
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 必要に応じて、分類レポートや混同行列の出力も可能
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_val, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, predictions))

Saving train_2.tsv to train_2 (1).tsv
Saving train_8.tsv to train_8 (2).tsv
Accuracy: 96.55%

Classification Report:
              precision    recall  f1-score   support

         acc       0.95      0.91      0.93        43
       class       1.00      1.00      1.00         1
        good       0.86      0.86      0.86         7
       unacc       0.98      0.99      0.98       119
       vgood       1.00      1.00      1.00         4

    accuracy                           0.97       174
   macro avg       0.96      0.95      0.95       174
weighted avg       0.97      0.97      0.97       174


Confusion Matrix:
[[ 39   0   1   3   0]
 [  0   1   0   0   0]
 [  1   0   6   0   0]
 [  1   0   0 118   0]
 [  0   0   0   0   4]]


# r4 r3から先頭行削除(idとかの行)

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1カラム目を削除（不要な情報が含まれている場合）
train_data = train_data.drop(columns=0)
val_data = val_data.drop(columns=0)

# インデックス0の行を削除
train_data = train_data.drop(index=0)
val_data = val_data.drop(index=0)
# train_dataの内容を再確認
print("1行目削除後のtrain_data:")
print(train_data)
print("1行目削除後のval_data:")
print(val_data)

# 説明変数と目的変数の分割（目的変数が新たに2カラム目となるので、列インデックス0）
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# カラム名をすべて文字列に変換
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 必要に応じて、分類レポートや混同行列の出力も可能
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_val, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, predictions))

Saving train_2.tsv to train_2.tsv
Saving train_8.tsv to train_8.tsv
1行目削除後のtrain_data:
         1      2      3  4  5      6     7
1    unacc    med   high  2  5    big   low
2      acc    med   high  3  4    med  high
3    unacc   high   high  5  5  small   low
4    unacc   high    med  3  2    big   low
5    unacc   high    low  5  4  small   low
..     ...    ...    ... .. ..    ...   ...
687    acc   high   high  2  4  small  high
688  unacc   high    low  2  5    med   med
689  unacc   high   high  5  4  small   med
690    acc   high   high  3  4  small  high
691  unacc  vhigh  vhigh  3  4  small  high

[691 rows x 7 columns]
1行目削除後のval_data:
         1      2      3  4  5      6     7
1      acc    low    low  3  5  small   med
2    unacc  vhigh  vhigh  4  2    med  high
3    unacc    low  vhigh  3  5  small   med
4    unacc  vhigh  vhigh  3  2  small   med
5    unacc    med   high  5  2    med   med
..     ...    ...    ... .. ..    ...   ...
169    acc  vhigh    med  4  5    me

# r６　直接パターン

In [9]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1カラム目を削除
train_data = train_data.drop(columns=0)
val_data = val_data.drop(columns=0)


# 説明変数と目的変数の分割
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 学習済みモデルの保存
joblib.dump(clf, 'decision_tree_model.pkl')

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 別のTSVファイルでモデルを評価するためのコード
def evaluate_model_on_new_data(new_file):
    # 新しいファイルのデータ読み込み
    new_data = pd.read_csv(new_file, sep='\t', header=None)
    new_data = new_data.drop(columns=0)  # 1カラム目を削除

    # 説明変数と目的変数の分割
    X_new = new_data.drop(columns=1)
    y_new = new_data[1]

    # Categorical feature encoding with alignment
    X_new = pd.get_dummies(X_new)
    X_new = X_new.reindex(columns=X_train.columns, fill_value=0)

    # 保存したモデルをロード
    clf_loaded = joblib.load('decision_tree_model.pkl')

    # 新しいデータでの予測と評価
    new_predictions = clf_loaded.predict(X_new)
    new_accuracy = accuracy_score(y_new, new_predictions)
    print(f"New Data Accuracy: {new_accuracy * 100:.2f}%")
    return new_predictions

# 使用例：evaluate_model_on_new_data('new_data.csv')

# 新しいデータファイルをアップロードして評価
# files.upload()  # 新しいデータファイルをアップロード
# evaluate_model_on_new_data('新しいデータファイル名をここに入力.tsv')

Saving train_2.tsv to train_2 (2).tsv
Saving train_8.tsv to train_8 (4).tsv
Accuracy: 96.55%


# モデルのダウンロード

In [10]:
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 例としてデータロードとモデル訓練を行います
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1カラム目を削除
train_data = train_data.drop(columns=0)
val_data = val_data.drop(columns=0)

# 説明変数と目的変数の分割
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 学習済みモデルの保存
model_filename = 'decision_tree_model.pkl'
joblib.dump(clf, model_filename)

# 学習済みモデルをローカルにダウンロード
files.download(model_filename)

Saving train_2.tsv to train_2 (3).tsv
Saving train_8.tsv to train_8 (5).tsv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# testtsvファイルにモデルを食わせる

In [11]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み、1行目と1列目を削除
test_file = 'test.tsv'
test_data = pd.read_csv(test_file, sep='\t', header=None, skiprows=1)
test_data = test_data.drop(columns=0)

# 説明変数と目的変数の分割
X_test = test_data.drop(columns=1)
y_test = test_data[1]

# Categorical feature encoding with alignment to the model's training data
X_test = pd.get_dummies(X_test)

# 学習済みモデルのロード
model_filename = 'decision_tree_model.pkl'
clf_loaded = joblib.load(model_filename)

# トレーニングデータと同じ列に揃える
# これを行わないと、新しいデータの特徴が異なっている場合にエラーになる可能性があります
X_test = X_test.reindex(columns=clf_loaded.feature_importances_.argsort()[::-1], fill_value=0)

# テストデータでの予測
predictions = clf_loaded.predict(X_test)

# 予測結果をCSVとして出力
result_df = pd.DataFrame({'Prediction': predictions})
result_filename = 'result.csv'
result_df.to_csv(result_filename, index=False)

# CSVファイルをローカルにダウンロード
files.download(result_filename)

Saving test.tsv to test (1).tsv


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 色々ためす

In [12]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み、1行目と1列目を削除
test_file = 'test.tsv'
test_data = pd.read_csv(test_file, sep='\t', header=None, skiprows=1)
test_data = test_data.drop(columns=0)

# 説明変数と目的変数の分割
X_test = test_data.drop(columns=1)
y_test = test_data[1]

# テストデータの先頭10行を標準出力
print("First 10 rows of the test data (after dropping first row and column):")
print(X_test.head(10))

# Categorical feature encoding with alignment to the model's training data
X_test = pd.get_dummies(X_test)

# 学習済みモデルのロード
model_filename = 'decision_tree_model.pkl'
clf_loaded = joblib.load(model_filename)

# 模型が訓練された特徴量の数に合わせる
# ここでは、すべての列をトレーニングデータと一致させる必要があります
# X_train_columns はトレーニングデータのカラムと定義されていたと仮定します
# X_test = X_test.reindex(columns=X_train_columns, fill_value=0)

# ここでは align ではなく、上記のように選択されたカラムの再調整を行うとします
# 但し、X_train_columns がどのようにキャプチャされるかが重要です。

# テストデータでの予測
predictions = clf_loaded.predict(X_test)

# 予測結果を標準出力
print("Predictions for the test data:")
print(predictions)

Saving test.tsv to test (2).tsv
First 10 rows of the test data (after dropping first row and column):
       2      3     4      5     6
0  vhigh      2  more    big   low
1  vhigh  5more     4  small  high
2   high      2     2  small   low
3    med      4     4    big   med
4   high      2     4    med   low
5    med      2     2  small  high
6    low      2  more  small   low
7   high      4  more  small  high
8    med      2  more    big   med
9    med      4     2  small   low


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 3_2
- 3_3
- 3_4
- 3_5more
- 4_more
- ...
Feature names seen at fit time, yet now missing:
- 2_buying
- 3_high
- 3_low
- 3_maint
- 3_med
- ...


特徴量チェック

In [13]:
# 学習済みモデルをロードする
clf_loaded = joblib.load('decision_tree_model.pkl')

# モデル自体が持つ特徴名属性を確認する例 (Scikit-learn の一部のモデルでは可能)
# 属性に依存する安全な方法は提供されていないため注意が必要
if hasattr(clf_loaded, 'feature_names_in_'):
    print(clf_loaded.feature_names_in_)
else:
    print("Feature names not stored in the model directly.")

['2_buying' '2_high' '2_low' '2_med' '2_vhigh' '3_high' '3_low' '3_maint'
 '3_med' '3_vhigh' '4_2' '4_3' '4_4' '4_5' '4_doors' '5_2' '5_4' '5_5'
 '5_persons' '6_big' '6_lug_boot' '6_med' '6_small' '7_high' '7_low'
 '7_med' '7_safety']


モデル作成処理のデバッグ

In [19]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# ローカルファイルをアップロード
uploaded = files.upload()

# ファイルの読み込み
train_file = 'train_8.tsv'
val_file = 'train_2.tsv'

train_data = pd.read_csv(train_file, sep='\t', header=None)
val_data = pd.read_csv(val_file, sep='\t', header=None)

# 1行目削除
train_data = train_data.drop(index=0)
val_data = val_data.drop(index=0)

# デバッグ
# 特定の行を表示
print("-------3line_data-------")
print(train_data.iloc[0:3])  # 例えば最初の3行を表示
# 先頭の5行を表示
print("-------train_data-------")
print(train_data.head())
print()
print("-------val_data-------")
print(val_data.head())

# 1カラム目を削除（不要な情報が含まれている場合）
train_data = train_data.drop(columns=0)
val_data = val_data.drop(columns=0)

# デバッグ
# 修正後先頭の5行を表示
print("-------drop_train_data-------")
print(train_data.head())
print()
print("-------drop_val_data-------")
print(val_data.head())

# 説明変数と目的変数の分割（目的変数が新たに2カラム目となるので、列インデックス0）
X_train = train_data.drop(columns=1)
y_train = train_data[1]

X_val = val_data.drop(columns=1)
y_val = val_data[1]

# デバッグ
# X_train.columnsを標準出力
print("X_train.columnsの中身:")
print(X_train.columns)
# リストとして出力
print("X_train.columnsのリスト表示:")
print(X_train.columns.tolist())
# X_val.columnsを標準出力
print("X_val.columnsの中身:")
print(X_val.columns)
# リストとして出力
print("X_val.columnsのリスト表示:")
print(X_val.columns.tolist())


# カラム名をすべて文字列に変換
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

# Categorical feature encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

# 対応する列を合わせる（片方にしかない列をカバー）
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

# 決定木モデルのインスタンス作成と訓練
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# 検証データでの予測と評価
predictions = clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 必要に応じて、分類レポートや混同行列の出力も可能
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_val, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, predictions))

Saving train_2.tsv to train_2 (7).tsv
Saving train_8.tsv to train_8 (9).tsv
-------3line_data-------
      0      1     2     3  4  5      6     7
1  1312  unacc   med  high  2  5    big   low
2   714    acc   med  high  3  4    med  high
3  1216  unacc  high  high  5  5  small   low
-------train_data-------
      0      1     2     3  4  5      6     7
1  1312  unacc   med  high  2  5    big   low
2   714    acc   med  high  3  4    med  high
3  1216  unacc  high  high  5  5  small   low
4    21  unacc  high   med  3  2    big   low
5   877  unacc  high   low  5  4  small   low

-------val_data-------
      0      1      2      3  4  5      6     7
1   383    acc    low    low  3  5  small   med
2  1442  unacc  vhigh  vhigh  4  2    med  high
3  1283  unacc    low  vhigh  3  5  small   med
4   627  unacc  vhigh  vhigh  3  2  small   med
5    59  unacc    med   high  5  2    med   med
-------drop_train_data-------
       1     2     3  4  5      6     7
1  unacc   med  high  2  5    bi

# r4モデルを使ってみる


In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files
import joblib

# ローカルファイルをアップロード
uploaded = files.upload()

# モデルの読み込み
model_filename = 'r4_decision_tree_model.pkl'
model = joblib.load(model_filename)  # joblibを使ってモデルをロード

# テストデータの読み込み
test_filename = 'test.tsv'
test_df = pd.read_csv(test_filename, sep='\t', header=None)

# 必要な前処理を行います（例：エンコーディング）
# ここでトレーニング時と同様の前処理を行います
# 例：テストデータのカテゴリ変数をone-hotエンコードするなど
# 実際には以下はトレーニング時の前処理に応じて変更してください
# test_df = pd.get_dummies(test_df)  # 仮の前処理
### やってたこと


# ユニークな値の置換を設定:
mapping = {'5more': 5, 'more': 5, 'med': 3}

# 安全な変換を実行:
test_df.iloc[:, 4] = test_df.iloc[:, 4].replace(mapping).astype(int, errors='ignore')
test_df.iloc[:, 5] = test_df.iloc[:, 5].replace(mapping).astype(int, errors='ignore')

# 変換されなかった値の処理:
print(test_df[test_df.iloc[:, 5].isna()])
print(test_df[test_df.iloc[:, 6].isna()])

# 1カラム目を削除（不要な情報が含まれている場合）
test_df = test_df.drop(columns=0)
# インデックス0の行を削除
test_df = test_df.drop(index=0)
print("1行目削除後のtest_df:")
print(test_df)

# カラム名をすべて文字列に変換
test_df.columns = test_df.columns.astype(str)
# Categorical feature encoding
test_df = pd.get_dummies(test_df)

### やってたことここまで
# 説明変数の抽出（目的変数がない前提）
X_test = test_df

# 目的変数の予測
predictions = model.predict(X_test)

# 結果の保存
result_df = pd.DataFrame(predictions, columns=['Predicted'])
result_filename = 'r4model_result.csv'
result_df.to_csv(result_filename, index=False)

print(f"Predictions saved to {result_filename}")

Saving r4_decision_tree_model.pkl to r4_decision_tree_model (1).pkl
Saving test.tsv to test (1).tsv
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
1行目削除後のtest_df:
         1      2      3  4      5     6
1     high  vhigh      2  5    big   low
2    vhigh  vhigh  5more  4  small  high
3      med   high      2  2  small   low
4      med    med      4  4    big   med
5     high   high      2  4      3   low
..     ...    ...    ... ..    ...   ...
860    low    low      3  2    big  high
861    med    low  5more  2    big   med
862  vhigh  vhigh      3  2    big   med
863   high  vhigh      4  4  small   low
864    low   high      3  2  small  high

[864 rows x 6 columns]


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 1_high
- 1_low
- 1_med
- 1_vhigh
- 3_2
- ...
Feature names seen at fit time, yet now missing:
- 3_high
- 3_low
- 3_med
- 3_vhigh
- 4_3
- ...


# r4モデルを無理やり使う_合わない場合0埋め

In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files
import joblib

# ローカルファイルをアップロード
uploaded = files.upload()

# モデルの読み込み
model_filename = 'r4_decision_tree_model.pkl'
model = joblib.load(model_filename)  # joblibを使ってモデルをロード

# テストデータの読み込み
test_filename = 'test.tsv'
test_df = pd.read_csv(test_filename, sep='\t', header=None)

# 必要な前処理を行います（例：エンコーディング）
# ここでトレーニング時と同様の前処理を行います
# 例：テストデータのカテゴリ変数をone-hotエンコードするなど
# 実際には以下はトレーニング時の前処理に応じて変更してください
# test_df = pd.get_dummies(test_df)  # 仮の前処理
### やってたこと


# ユニークな値の置換を設定:
mapping = {'5more': 5, 'more': 5, 'med': 3}

# 安全な変換を実行:
test_df.iloc[:, 4] = test_df.iloc[:, 4].replace(mapping).astype(int, errors='ignore')
test_df.iloc[:, 5] = test_df.iloc[:, 5].replace(mapping).astype(int, errors='ignore')

# 変換されなかった値の処理:
print(test_df[test_df.iloc[:, 5].isna()])
print(test_df[test_df.iloc[:, 6].isna()])

# 1カラム目を削除（不要な情報が含まれている場合）
test_df = test_df.drop(columns=0)
# インデックス0の行を削除
test_df = test_df.drop(index=0)
print("1行目削除後のtest_df:")
print(test_df)

# カラム名をすべて文字列に変換
test_df.columns = test_df.columns.astype(str)
# Categorical feature encoding
test_df = pd.get_dummies(test_df)

### やってたことここまで

# 特徴のエンコードをトレーニングと一致させる
# 仮にOne-Hot Encodingを行った場合
test_df_encoded = pd.get_dummies(test_df)

# トレーニング時に使用した特徴名を取得
feature_names = model.feature_names_in_

# テストデータを再整形して、訓練時のすべての特徴名を持たせる
# 存在しない特徴は0で埋めます
X_test = test_df_encoded.reindex(columns=feature_names, fill_value=0)

# 目的変数の予測
predictions = model.predict(X_test)

# インデックス列(1から始まる)を追加して結果の保存
result_df = pd.DataFrame({
    'Index': range(1, len(predictions) + 1),  # 1から始まる連番
    'Predicted': predictions
})

# 結果の保存
result_filename = 'r4_decision_tree_model_result.csv'
result_df.to_csv(result_filename, index=False, header=False)

print(f"Predictions saved to {result_filename}")
files.download('r4_decision_tree_model_result.csv')

Saving r4_decision_tree_model.pkl to r4_decision_tree_model (5).pkl
Saving test.tsv to test (5).tsv
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
1行目削除後のtest_df:
         1      2      3  4      5     6
1     high  vhigh      2  5    big   low
2    vhigh  vhigh  5more  4  small  high
3      med   high      2  2  small   low
4      med    med      4  4    big   med
5     high   high      2  4      3   low
..     ...    ...    ... ..    ...   ...
860    low    low      3  2    big  high
861    med    low  5more  2    big   med
862  vhigh  vhigh      3  2    big   med
863   high  vhigh      4  4  small   low
864    low   high      3  2  small  high

[864 rows x 6 columns]
Predictions saved to r4_decision_tree_model_result.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# r4でID無理やり合わせる

In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files
import joblib

# ローカルファイルをアップロード
uploaded = files.upload()

# モデルの読み込み
model_filename = 'r4_decision_tree_model.pkl'
model = joblib.load(model_filename)  # joblibを使ってモデルをロード

# テストデータの読み込み
test_filename = 'test.tsv'
test_df = pd.read_csv(test_filename, sep='\t', header=None)

# test.tsvの1行目の2列目以降の数値データを取得
#first_column_data = test_df.iloc[1:, 0]
# 1列目の2行目以降のデータを取得
index_data = test_df.iloc[1:, 0].reset_index(drop=True)


# 結果を表示
print("-----")
print(first_column_data)

# 必要な前処理を行います（例：エンコーディング）
# ここでトレーニング時と同様の前処理を行います
# 例：テストデータのカテゴリ変数をone-hotエンコードするなど
# 実際には以下はトレーニング時の前処理に応じて変更してください
# test_df = pd.get_dummies(test_df)  # 仮の前処理
### やってたこと


# ユニークな値の置換を設定:
mapping = {'5more': 5, 'more': 5, 'med': 3}

# 安全な変換を実行:
test_df.iloc[:, 4] = test_df.iloc[:, 4].replace(mapping).astype(int, errors='ignore')
test_df.iloc[:, 5] = test_df.iloc[:, 5].replace(mapping).astype(int, errors='ignore')

# 変換されなかった値の処理:
print(test_df[test_df.iloc[:, 5].isna()])
print(test_df[test_df.iloc[:, 6].isna()])

# 1カラム目を削除（不要な情報が含まれている場合）
test_df = test_df.drop(columns=0)
# インデックス0の行を削除
test_df = test_df.drop(index=0)
print("1行目削除後のtest_df:")
print(test_df)

# カラム名をすべて文字列に変換
test_df.columns = test_df.columns.astype(str)
# Categorical feature encoding
test_df = pd.get_dummies(test_df)

### やってたことここまで

# 特徴のエンコードをトレーニングと一致させる
# 仮にOne-Hot Encodingを行った場合
test_df_encoded = pd.get_dummies(test_df)

# トレーニング時に使用した特徴名を取得
feature_names = model.feature_names_in_

# テストデータを再整形して、訓練時のすべての特徴名を持たせる
# 存在しない特徴は0で埋めます
X_test = test_df_encoded.reindex(columns=feature_names, fill_value=0)

# 目的変数の予測
predictions = model.predict(X_test)

# 結果のDataFrameを作成
result_df = pd.DataFrame({
    'Index': index_data,
    'Predicted': predictions
})


# DataFrame用に整形し、Index列に1を追加
#first_column_df = pd.DataFrame([[1] + first_column_data], columns=result_df.columns)
# 新しいDataFrameに代入
#print("--aa---")
#print(first_column_data)
#first_column_df = pd.DataFrame(first_column_data, columns=['First_Column'])

# 結果DataFrameの先頭に挿入
#combined_df = pd.concat([first_column_df, result_df], ignore_index=True)

# CSVに保存（ヘッダー無し）
result_filename = 'r4_decision_tree_model_result.csv'
result_df.to_csv(result_filename, index=False, header=False)

print(f"Predictions saved to {result_filename}")
files.download('r4_decision_tree_model_result.csv')

Saving r4_decision_tree_model.pkl to r4_decision_tree_model (19).pkl
Saving test.tsv to test (19).tsv
-----
1         1
2         2
3         4
4         5
5         6
       ... 
860    1718
861    1719
862    1720
863    1723
864    1727
Name: 0, Length: 864, dtype: object
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
1行目削除後のtest_df:
         1      2      3  4      5     6
1     high  vhigh      2  5    big   low
2    vhigh  vhigh  5more  4  small  high
3      med   high      2  2  small   low
4      med    med      4  4    big   med
5     high   high      2  4      3   low
..     ...    ...    ... ..    ...   ...
860    low    low      3  2    big  high
861    med    low  5more  2    big   med
862  vhigh  vhigh      3  2    big   med
863   high  vhigh      4  4  small   low
864    low   high      3  2  small  high

[864 rows x 6 columns]
Predictions saved to r4_decision_tree_model_result.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>